In [33]:
from numpy import *

In [34]:
def stationary_reactive(p1, p2, delta):
    M = array(
        [
            [
                p1[1] * p2[1],
                p1[1] * (1 - p2[1]),
                (1 - p1[1]) * p2[1],
                (1 - p1[1]) * (1 - p2[1]),
            ],
            [
                p1[2] * p2[1],
                p1[2] * (1 - p2[1]),
                (1 - p1[2]) * p2[1],
                (1 - p1[2]) * (1 - p2[1]),
            ],
            [
                p1[1] * p2[2],
                p1[1] * (1 - p2[2]),
                (1 - p1[1]) * p2[2],
                (1 - p1[1]) * (1 - p2[2]),
            ],
            [
                p1[2] * p2[2],
                p1[2] * (1 - p2[2]),
                (1 - p1[2]) * p2[2],
                (1 - p1[2]) * (1 - p2[2]),
            ],
        ]
    )

    v0 = array(
        [
            p1[0] * p2[0],
            p1[0] * (1 - p2[0]),
            (1 - p1[0]) * p2[0],
            (1 - p1[0]) * (1 - p2[0]),
        ]
    )

    return (1 - delta) * v0 @ linalg.inv((eye(4) - delta * M))

In [35]:
population = [[0, 0, 0, 100]]

In [41]:
import numpy as np

In [81]:
population = np.random.random((100, 3))

In [103]:
population = np.zeros((100, 3))

In [104]:
Mu, c = np.unique(population, axis=0, return_counts=True)

In [101]:
for p in Mu:
    print(p[1])

0.0


In [96]:
c

array([100])

In [112]:
def cooperation(population, delta):
    Mu, counts = unique(population, axis=0, return_counts=True)
    coop = 0
    for i, p in enumerate(Mu):
        print(p)
        for j, q in enumerate(Mu):
            v = stationary_reactive(p, q, delta)
            coop += v[1] + v[2] * counts[j] - int(i == j)

    return coop / sum(counts)


In [176]:
delta = 0.999

In [122]:
cooperation(population, delta)

-0.01

In [191]:
population = np.array([[0, 0, 0]] * 10 + [[1, 1, 1]] * 5 + [[0, 1, 0]] * 4 + [[1, 0, 0]] * 1)

In [192]:
population

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0]])

In [206]:
v = stationary_reactive([1, 0, 0], [1, 0, 0], delta)
v[0] + v[1]

0.0010000000000000009

In [231]:
population = np.random.random((50, 3))
population

array([[0.81740525, 0.8399056 , 0.73494736],
       [0.98349908, 0.65852988, 0.18701107],
       [0.7628261 , 0.52483867, 0.84054356],
       [0.41362593, 0.64118857, 0.73528022],
       [0.87132849, 0.51243269, 0.76625232],
       [0.96148414, 0.25607347, 0.6668701 ],
       [0.9212705 , 0.48448697, 0.05820347],
       [0.90643825, 0.85572503, 0.48523087],
       [0.06811712, 0.17121679, 0.86908016],
       [0.35087503, 0.46612688, 0.02270656],
       [0.8361916 , 0.93820736, 0.90358379],
       [0.38675835, 0.72938095, 0.92636367],
       [0.40137801, 0.60446375, 0.08127874],
       [0.79291006, 0.90107187, 0.40035308],
       [0.81010427, 0.79948445, 0.57274616],
       [0.72416911, 0.02497506, 0.75476844],
       [0.54960921, 0.4182098 , 0.65325719],
       [0.89645456, 0.05035718, 0.24236054],
       [0.18529522, 0.5991691 , 0.41202868],
       [0.24145877, 0.66017583, 0.04517691],
       [0.00343473, 0.53208759, 0.64799668],
       [0.71317371, 0.80901411, 0.11455812],
       [0.

In [242]:
population_B = np.zeros((50, 3))

In [243]:
def cooperation(population, delta):
    Mu, counts = unique(population, axis=0, return_counts=True)
    coop = 0
    for i, p in enumerate(Mu):
        for j, q in enumerate(Mu):
            v = stationary_reactive(p, q, delta)
            coop += ((v[0] + v[1]) * (counts[j]) / sum(counts))

    return coop

In [248]:
Mu, counts = unique(population, axis=0, return_counts=True)

In [250]:
def cooperation(population, delta, N):
    Mu, counts = unique(population, axis=0, return_counts=True)
    unique_rows = len(counts)
    coop = zeros(unique_rows)
    for i, me in enumerate(Mu):
        player_coop = zeros(unique_rows)
        for other in Mu:
            v = stationary_reactive(me, other, delta)
            player_coop[i] = v[0] + v[1]

        coop[i] = sum(player_coop * counts / sum(counts))
    return mean(coop)

50

In [246]:
cooperation(population_B, delta)

0.0

In [247]:
cooperation(population, delta)

27.15151788686418

In [209]:
population[-1]

array([1, 0, 0])

In [172]:
population.mean(0)

array([0.58381198, 0.52892058, 0.49301   ])